In [13]:
!pip install torch

In [3]:
!pip install pytorch-tabnet


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import accuracy_score


In [7]:
data =pd.read_csv('cardio_train.csv',delimiter=";")

In [8]:
data=pd.DataFrame(data)
data

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,99993,19240,2,168,76.0,120,80,1,1,1,0,1,0
69996,99995,22601,1,158,126.0,140,90,2,2,0,0,1,1
69997,99996,19066,2,183,105.0,180,90,3,1,0,1,0,1
69998,99998,22431,1,163,72.0,135,80,1,2,0,0,0,1


In [9]:
data=data.drop(columns=['id'])

In [10]:
X = data.drop(columns=["cardio"])
y = data["cardio"]

In [30]:
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

X_train, X_valid, y_train, y_valid = train_test_split(X.values, y.values, test_size=0.2, random_state=42)

In [31]:
# tabnet initialisation
import torch
clf = TabNetClassifier(
    n_d=8,
    n_a=8,
    n_steps=3,
    gamma=1.3,
    lambda_sparse=1e-3,
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    mask_type="sparsemax"
)


/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


In [32]:
#train the model of the cardiotrain dataset
clf.fit(
    X_train=X_train,
    y_train=y_train,
    eval_set=[(X_valid, y_valid)],
    eval_name=["val"],
    eval_metric=["accuracy"],
    max_epochs=100,    #since size of dataset <10000 max epoches can be between 100 and 200 for better results
    patience=10,
    batch_size=256,
    virtual_batch_size=128,
    num_workers=0,
    drop_last=False
)


epoch 0  | loss: 0.58701 | val_accuracy: 0.67279 |  0:00:06s
epoch 1  | loss: 0.56307 | val_accuracy: 0.67714 |  0:00:11s
epoch 2  | loss: 0.55739 | val_accuracy: 0.68171 |  0:00:17s
epoch 3  | loss: 0.55672 | val_accuracy: 0.70364 |  0:00:23s
epoch 4  | loss: 0.55312 | val_accuracy: 0.66436 |  0:00:29s
epoch 5  | loss: 0.55382 | val_accuracy: 0.691   |  0:00:35s
epoch 6  | loss: 0.55529 | val_accuracy: 0.70571 |  0:00:41s
epoch 7  | loss: 0.55252 | val_accuracy: 0.70421 |  0:00:46s
epoch 8  | loss: 0.55312 | val_accuracy: 0.71521 |  0:00:54s
epoch 9  | loss: 0.55207 | val_accuracy: 0.60521 |  0:01:00s
epoch 10 | loss: 0.55409 | val_accuracy: 0.67771 |  0:01:08s
epoch 11 | loss: 0.55249 | val_accuracy: 0.66279 |  0:01:14s
epoch 12 | loss: 0.55238 | val_accuracy: 0.68586 |  0:01:20s
epoch 13 | loss: 0.55199 | val_accuracy: 0.64807 |  0:01:25s
epoch 14 | loss: 0.55143 | val_accuracy: 0.68736 |  0:01:32s
epoch 15 | loss: 0.54906 | val_accuracy: 0.701   |  0:01:37s
epoch 16 | loss: 0.54928

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [33]:
# evaluation

# Make predictions on the validation set
y_pred = clf.predict(X_valid)

# Calculate accuracy
accuracy = accuracy_score(y_valid, y_pred)
print("Validation Accuracy:", accuracy)


Validation Accuracy: 0.7152142857142857


In [34]:
# Feature importance (interpretability)
import numpy as np

feature_importances = clf.feature_importances_
feature_names = X.columns
print("Feature Importances:")
for name, importance in zip(feature_names, feature_importances):
    print(f"{name}: {importance:.4f}")


Feature Importances:
age: 0.3036
gender: 0.0007
height: 0.0000
weight: 0.1298
ap_hi: 0.3648
ap_lo: 0.0650
cholesterol: 0.1165
gluc: 0.0000
smoke: 0.0000
alco: 0.0103
active: 0.0094


inistially we encoutnred issues with emplying tab net , we decided to retrain the model wioth different epoch size cause it didnt make sense that the smoking and activity facotr had little impact on wheather or not the person has cardiovasculair issues, the resilt were the same thoguh so we kept the parameters as they are

In [35]:
torch.save(clf, 'tabnet__model.pth')

code snippet of how import and employ the model

In [36]:
clf_loaded = torch.load('tabnet__model.pth')

<ipython-input-36-da15117260b2>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  clf_loaded = torch.load('tabnet__model.pth')
